In [1]:
import  requests
import json

In [38]:
API_CREDENTIALS = 'oxford_api_credentials.txt'
with open(API_CREDENTIALS) as f:
    creds = f.read()

In [39]:
api_creds = {}
creds.split('\n')
api_creds['Accept'] = creds.split('\n')[0].split(':')[1].strip()
api_creds['app_id'] = creds.split('\n')[1].split(':')[1].strip()
api_creds['app_key'] = creds.split('\n')[2].split(':')[1].strip()
api_creds['url'] = creds.split('\n')[3].split(':', 1)[1].strip()

api_creds

{'Accept': 'application/json',
 'app_id': '8beabadc',
 'app_key': '4d85f3e2e7cd293da9a811f156c99841',
 'url': 'https://od-api.oxforddictionaries.com/api/v2'}

In [36]:
def json_load(res):
  t = json.dumps(res.json())
  return json.loads(t)

headers = {
        "Accept": api_creds['Accept'],
        "app_id": api_creds['app_id'],
        "app_key": api_creds['app_key']
    }
word_id = 'abusive'
url_entries = api_creds['url'] + 'entries/en/' + word_id + '?strictMatch=true'
res_entries = requests.get(url_entries, headers = {'app_id' : headers['app_id'], 'app_key' : headers['app_key']})

senses_examples = json_load(res_entries)

In [37]:
senses_examples['results'][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['examples'][0]['text']

'the goalkeeper was sent off for using abusive language'

In [111]:
def json_load(res):
  t = json.dumps(res.json())
  return json.loads(t)

word_id = 'Washington'
url_entries = 'https://od-api.oxforddictionaries.com/api/v2/entries/en/' + word_id + '?strictMatch=true'
res_entries = requests.get(url_entries, headers = {'app_id' : headers['app_id'], 'app_key' : headers['app_key']})

json_load(res_entries)

{'id': 'washington',
 'metadata': {'operation': 'retrieve',
  'provider': 'Oxford University Press',
  'schema': 'RetrieveEntry'},
 'results': [{'id': 'Washington',
   'language': 'en-gb',
   'lexicalEntries': [{'entries': [{'grammaticalFeatures': [{'id': 'proper',
         'text': 'Proper',
         'type': 'Properness'}],
       'pronunciations': [{'audioFile': 'https://audio.oxforddictionaries.com/en/mp3/washington_1_gb_1.mp3',
         'dialects': ['British English'],
         'phoneticNotation': 'IPA',
         'phoneticSpelling': 'ˈwɒʃɪŋt(ə)n'}],
       'senses': [{'definitions': ['a state of the north-western US, on the coast of the Pacific Ocean; population 6,549,224 (est. 2008); capital, Olympia. It became the 42nd state in 1889.'],
         'id': 'm_en_gbus1142220.006',
         'semanticClasses': [{'id': 'us_state', 'text': 'Us_State'}],
         'shortDefinitions': ['state of north-western US, on Pacific coast']},
        {'definitions': ['the capital of the US; population 

In [131]:
def json_load(res):
  t = json.dumps(res.json())
  return json.loads(t)

def prepare_api(word:str):
    headers = {
        "Accept": "application/json",
        "app_id": "8beabadc",
        "app_key": "4d85f3e2e7cd293da9a811f156c99841"
    }
    word_id = word
    url_entries = 'https://od-api.oxforddictionaries.com/api/v2/entries/en/' + word_id + '?strictMatch=true'
    res_entries = requests.get(url_entries, headers = {'app_id' : headers['app_id'], 'app_key' : headers['app_key']})
    url_sentences = 'https://od-api.oxforddictionaries.com/api/v2/sentences/en/' + word_id + '?strictMatch=true'
    res_sentences = requests.get(url_sentences, headers = {'app_id' : headers['app_id'], 'app_key' : headers['app_key']})

    senses_examples = json_load(res_entries)
    sentences_examples = json_load(res_sentences)

    return senses_examples, sentences_examples

def run_words(word:str):
    from itertools import chain
    senses_examples, sentences_examples = prepare_api(word)

    try:
        senses_examples['results']
    except KeyError:
        raise ValueError(
            'No resutls for senses'
        )
    try:
        sentences_examples['results']
    except KeyError:
        raise ValueError(
            'No resutls for senteces'
        )

    api_call_senses = senses_examples['results'][0]['lexicalEntries'][0]['entries'][0]['senses']
    sentences = sentences_examples['results'][0]['lexicalEntries'][0]['sentences']
    senses = []
    sense_with_examples = {}
    diff_sense_ids = []


    for el in sentences:
        diff_sense_ids.append(el['senseIds'][0])

    sense_ids = set(diff_sense_ids)

    def search(id):
        return list(sent['text'] for sent in sentences if sent['senseIds'][0] == id)

    for idx, sens in enumerate(api_call_senses):
        try:
            sense_with_examples['word'] = word
            sense_with_examples['sense'] = sens['id']
            sense_with_examples['definition'] = sens['definitions'][0]
            examples_for_senses = list(ex['text'] for ex in sens['examples'])

            if sens['id'] in list(sense_ids):
                examples_sense = search(sens['id'])
                sense_with_examples['examples'] = list(chain(examples_sense, examples_for_senses))
        except KeyError:
            raise ValueError(
                'No examples for the word: {}'.format(word)
            )

        senses.append(sense_with_examples.copy())

    return senses

In [118]:
senses = run_words('people')

In [132]:
senses

[{'sense': 'm_en_gbus0763900.006',
  'definition': 'human beings in general or considered collectively',
  'examples': ["He's a very strong personality, but he talks to people as human beings and he's very honest.",
   'The Home Office had to treat these people as decent human beings and provide extra resources.',
   'We may well decide that it was the most evil act ever perpetrated by human beings on fellow people.',
   "We don't have nearly the amount of litter because people in general take pride in their city.",
   'It is high among the reasons why people consult general practitioners and neurologists.',
   'You can count the number of people at most general openings on your fingers and toes.',
   'If so, was his stringent demand only for disciples, or was it intended for people in general?',
   'Neither do I have a problem in general with people who wish to follow religious beliefs.',
   'As I grew older, my imaginary friends took on the personas of real living people.',
   'In ge

In [175]:
## Read words and load into json file:
TARGET_WORDS_PATH = '../data/target_words/target.txt'
with open(TARGET_WORDS_PATH) as f:
    full_text = f.read()

all_words = []
for word in full_text.split('\n'):
    try:
        out_dict = run_words(word)
        all_words.append(out_dict)
    except ValueError:
        continue

all_words

KeyboardInterrupt: 

In [169]:
def look_for_poly():
    for word in all_words:
        if len(word) > 4:
            yield word[0]['word']

# with open('../News-data-project/target words/senses_oxford_api.txt', 'w') as f:
#     json.dump(list(look_for_poly()), f, indent=4)

In [172]:
list(look_for_poly())

['state', 'right', 'around', 'black', 'force', 'interest', 'support', 'charge']

In [105]:
from nltk import pos_tag, word_tokenize

def search_word_tag(tagged_words:list, word_s:str):
    return next(word for word in tagged_words if word[0] == word_s)

for sens in senses:
    for ex in sens['examples']:
        tokens = word_tokenize(ex)
        tags = pos_tag(tokens)
        print(tags)
        print('\n')
        # print(search_word_tag(tags, 'love'))


[('That', 'DT'), ('year', 'NN'), ('he', 'PRP'), ('moved', 'VBD'), ('to', 'TO'), ('London', 'NNP'), ('but', 'CC'), ('his', 'PRP$'), ('love', 'NN'), ('for', 'IN'), ('Wales', 'NNP'), ('was', 'VBD'), ('strong', 'JJ'), ('and', 'CC'), ('he', 'PRP'), ('eventually', 'RB'), ('settled', 'VBD'), ('permanently', 'RB'), ('there', 'RB'), ('.', '.')]


[('His', 'PRP$'), ('love', 'NN'), ('for', 'IN'), ('children', 'NNS'), ('and', 'CC'), ('affection', 'NN'), ('for', 'IN'), ('the', 'DT'), ('sick', 'NN'), ('have', 'VBP'), ('endeared', 'VBN'), ('him', 'PRP'), ('to', 'TO'), ('all', 'DT'), ('.', '.')]


[('My', 'PRP$'), ('brother', 'NN'), (',', ','), ('and', 'CC'), ('his', 'PRP$'), ('real', 'JJ'), (',', ','), ('strong', 'JJ'), ('love', 'NN'), ('for', 'IN'), ('me', 'PRP'), ('that', 'DT'), ('was', 'VBD'), ('able', 'JJ'), ('to', 'TO'), ('pull', 'VB'), ('me', 'PRP'), ('back', 'RB'), ('into', 'IN'), ('the', 'DT'), ('world', 'NN'), ('I', 'PRP'), ('know', 'VBP'), ('.', '.')]


[('Each', 'DT'), ('one', 'CD'), ('is'

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aymanehachcham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
